In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def pad_and_concat(sequences):  # sequences shape: [batch_size, len, dims...] -> ([batch_size, maxlen, dims...], [len])
    arrays = [np.asarray(seq) for seq in sequences]
    lengths = np.asarray([array.shape[0] for array in arrays], dtype=np.int32)
    maxlen = np.max(lengths)
    arrays = [np.pad(array, [(0, maxlen - array.shape[0]), (0, 0)], 'constant', constant_values=0) for array in arrays]
    return np.asarray(arrays), lengths
    

In [3]:
cnf, lengths = pad_and_concat(
    [[[1, -2], [2, 1]],
     [[-2, -1], [1, -2]],
     [[-1, -1], [-2, -2],
      #[-1, -2]
     ]
       ])
sol = np.asarray([[1, 1], [-1, -1], 
                  [-1, -1]
                 ])

In [4]:
print(cnf)
print(lengths)

[[[ 1 -2]
  [ 2  1]]

 [[-2 -1]
  [ 1 -2]]

 [[-1 -1]
  [-2 -2]]]
[2 2 2]


In [5]:
VARIABLE_NUM = 2
EMBEDDING_SIZE = 8
CLAUSE_SIZE = 2
LSTM_STATE_SIZE = 8
batch_size = 3

In [6]:
def assert_shape(matrix, shape: list):
    act_shape = matrix.get_shape().as_list()
    assert act_shape == shape, "got shape {}, expected {}".format(act_shape, shape)

In [7]:
class Graph:
    def __init__(self):
        self.inputs = tf.placeholder(tf.int32, shape=(batch_size, None, CLAUSE_SIZE), name='inputs')
        #self.lengths = tf.placeholder(tf.int32, shape=(batch_size,), name='lengths')
        self.labels = tf.placeholder(tf.float32, shape=(batch_size, VARIABLE_NUM), name='labels')
        
        vars_ = tf.abs(self.inputs) - 1
        signs = tf.cast(tf.sign(self.inputs), tf.float32)  # shape: [batch_size, None, CLAUSE_SIZE]

        embeddings = tf.Variable(tf.random_uniform([VARIABLE_NUM, EMBEDDING_SIZE], -1., 1), name='embeddings')

        var_embeddings = tf.nn.embedding_lookup(embeddings, vars_)
        # var_embeddings shape: [batch_size, None, CLAUSE_SIZE, EMBEDDING_SIZE]
        
        clause_preembeddings = tf.concat(
            [tf.reshape(var_embeddings, [batch_size, -1, CLAUSE_SIZE * EMBEDDING_SIZE]), 
             signs],
            axis=2)
        
        PREEMBEDDING_SIZE = EMBEDDING_SIZE * CLAUSE_SIZE + CLAUSE_SIZE
        assert_shape(clause_preembeddings, 
                     [batch_size, None, PREEMBEDDING_SIZE])
        
        clause_w = tf.Variable(tf.random_normal(
            [PREEMBEDDING_SIZE, EMBEDDING_SIZE]), name='clause_w')
        clause_b = tf.Variable(tf.random_normal([EMBEDDING_SIZE]), name='clause_b')
        clause_embeddings = tf.reshape(tf.sigmoid(
            tf.reshape(clause_preembeddings, [-1, PREEMBEDDING_SIZE]) @ clause_w + clause_b), 
                                       [batch_size, -1, EMBEDDING_SIZE])
        # shape: [batch_size, None, EMBEDDING_SIZE]
        
        lstm = tf.contrib.rnn.BasicLSTMCell(LSTM_STATE_SIZE)
        hidden_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        current_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        state = hidden_state, current_state
        
        _, lstm_final_state = tf.nn.dynamic_rnn(lstm, clause_embeddings, dtype=tf.float32, 
                                               #sequence_length=self.lengths
                                               )
        formula_embedding = lstm_final_state.h
        # for timestep in tf.unstack(clause_embeddings, num=LEN, axis=1):
        #    formula_embedding, state = lstm(timestep, state)
            
        assert_shape(formula_embedding, [batch_size, LSTM_STATE_SIZE])
            
        softmax_w = tf.Variable(tf.random_normal([LSTM_STATE_SIZE, VARIABLE_NUM]), name='softmax_w')
        softmax_b = tf.Variable(tf.random_normal([VARIABLE_NUM]), name='softmax_b')
        
        self.logits = formula_embedding @ softmax_w + softmax_b
        self.loss = tf.losses.sigmoid_cross_entropy((self.labels + 1) / 2, self.logits) 
        self.probabilities = tf.sigmoid(self.logits) * 2 - 1

model = Graph()

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    logits, probs = sess.run([model.logits, model.probabilities], feed_dict={
        model.inputs: cnf,
        # model.lengths: lengths
    })
    
print(logits)
print(probs)

[[ 0.7092738 -1.0156794]
 [ 0.4476912 -1.0542433]
 [ 0.5607948 -1.0825043]]
[[ 0.3404814  -0.4682603 ]
 [ 0.22018027 -0.48317778]
 [ 0.27327287 -0.49393523]]


In [9]:
with tf.Session() as sess:
    train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(model.loss)
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(train_op, feed_dict={
             model.inputs: cnf,
            model.labels: sol,
            #model.lengths: lengths
        })
    probs = sess.run(model.probabilities, feed_dict={
        model.inputs: cnf,
        #model.lengths: lengths
    })
    
probs

array([[ 0.9966111 ,  0.99328256],
       [-0.99948275, -0.99910104],
       [-0.9995619 , -0.99890375]], dtype=float32)